# Загрузка состава индекса IMOEX

In [1]:
import requests
import pandas as pd

Загрузка весов инструментов в индексе

In [2]:
analytics_url = \
'http://iss.moex.com/iss/statistics/engines/stock/markets/index/analytics/IMOEX.json?start={i}&analytics.columns={columns}'

columns = ['ticker', 'weight']

res = requests.get(analytics_url.format(i=0, columns=','.join(columns)))
res_json = res.json()

pagesize_index = res_json['analytics.cursor']['columns'].index('PAGESIZE')
pagesize = res_json['analytics.cursor']['data'][0][pagesize_index]

total_index = res_json['analytics.cursor']['columns'].index('TOTAL')
total = res_json['analytics.cursor']['data'][0][total_index]

In [3]:
analitics_rows = []
for i in range(0, total, pagesize):
    res = requests.get(analytics_url.format(i=i, columns=','.join(columns)))
    res_json = res.json()
    analitics_rows += res_json['analytics']['data']

In [4]:
analytics_data = pd.DataFrame(analitics_rows, columns=columns)
analytics_data.head()

,ticker,weight
0,AFKS,0.40
1,AFLT,0.40
2,ALRS,1.85
3,CBOM,0.29
4,CHMF,1.30


Загрузка актуальных цен закрытия и размеров лотов

In [5]:
securities_metadata_url = \
'https://iss.moex.com/iss/engines/stock/markets/shares/boards/TQBR/securities/metadata.json'

securities_url = \
'https://iss.moex.com/iss/engines/stock/markets/shares/boards/TQBR/securities.json?securities={securities}'

res = requests.get(securities_metadata_url)
res_json = res.json()

lotsize_index = res_json['securities']['columns'].index('LOTSIZE')
lastprice_index = res_json['marketdata']['columns'].index('LAST')

In [6]:
lotsize_rows = []
lastprice_rows = []
for i in range(0, total, 10):
    securities = ','.join(analytics_data.ticker[i:i+10])
    res = requests.get(securities_url.format(securities=securities))
    res_json = res.json()
    for securities_row in res_json['securities']['data']:
        lotsize_rows.append(securities_row[lotsize_index])
    for marketdata_row in res_json['marketdata']['data']:
        lastprice_rows.append(marketdata_row[lastprice_index])

Состав индекса

In [7]:
analytics_data['lotsize'] = lotsize_rows
analytics_data['lastprice'] = lastprice_rows
analytics_data['weight'] = analytics_data['weight'] * 0.01
analytics_data

,ticker,weight,lotsize,lastprice
0,AFKS,0.0040,100,15.04700
1,AFLT,0.0040,10,105.04000
2,ALRS,0.0185,10,89.30000
3,CBOM,0.0029,100,5.90100
4,CHMF,0.0130,1,937.40000
5,DSKY,0.0022,10,105.56000
6,FEES,0.0039,10000,0.20602
7,FIVE,0.0203,1,2202.00000
8,GAZP,0.1377,10,251.90000
9,GMKN,0.0667,1,19878.00000


# Формирование портфеля

Входные данные:

$T$ - объем инвестиций в рублях \
$a$ - вектор весов инструментов в индексе\
$p$ - вектор актуальных цен закрытия \
$l$ - вектор количества бумаг в одном лоте (размеры лотов) \
$n$ - количество инструментов в индексе

Требуется найти:

$x$ - вектор весов инструментов в портфеле \
$y$ - вектор количества лотов

Решаем задачу оптимизации с ограничениями

$$f(x) = \sum_i^n (x_i - a_i)^2 \to \min$$

$$\begin{cases}
\sum_i^n x_i \leq 1 \\
\forall i: x_i > 0 \\
\forall i: y_i = T x_i\frac{1}{p_i} \frac{1}{l_i}\\
\forall i: y_i \in \mathbb Z
\end{cases}$$

In [8]:
import cvxpy as cp
import numpy as np

In [9]:
total = 1000000
weight = np.array(analytics_data.weight)
price = np.array(analytics_data.lastprice)
lotsize = np.array(analytics_data.lotsize)

x_weight = cp.Variable(analytics_data.shape[0], nonneg=True)
y_count = cp.Variable(analytics_data.shape[0], integer=True)

obj = cp.Minimize(cp.sum_squares(x_weight - weight))
constr1 = (y_count == total * x_weight / price / lotsize)
constr2 = cp.sum(x_weight) <= 100

problem = cp.Problem(obj, [constr1, constr2])
problem.solve()
problem.status

'optimal_inaccurate'

Стоимость портфеля

In [10]:
round((y_count.value * analytics_data.lotsize * analytics_data.lastprice).sum(), 2)

989720.0

Состав портфеля

In [11]:
potfolio = pd.DataFrame(analytics_data[['ticker']])
potfolio['weight'] = x_weight.value.round(4)
potfolio['count'] = y_count.value * analytics_data.lotsize
potfolio['total'] = y_count.value * analytics_data.lotsize * analytics_data.lastprice
potfolio

,ticker,weight,count,total
0,AFKS,0.0045,300.0,4514.1
1,AFLT,0.0042,40.0,4201.6
2,ALRS,0.0188,210.0,18753.0
3,CBOM,0.0030,500.0,2950.5
4,CHMF,0.0131,14.0,13123.6
5,DSKY,0.0021,20.0,2111.2
6,FEES,0.0041,20000.0,4120.4
7,FIVE,0.0198,9.0,19818.0
8,GAZP,0.1385,550.0,138545.0
9,GMKN,0.0596,3.0,59634.0
